In [1]:
import os

import numpy as np
import regex as re
import pandas as pd

In [12]:
from collections import Counter

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package punkt to /home/varsey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/varsey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /home/varsey/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [3]:
from rapidfuzz import process as proc
from rapidfuzz import fuzz

In [271]:
from tqdm import tqdm

In [4]:
from stemmer_aze.stemmer import Stemmer

st = Stemmer()

In [5]:
# Загружаем данные
df_incoming = pd.read_excel(f'{os.getcwd()}/data/drl-data.xlsx', sheet_name='Inventory Out')

In [6]:
# Загружаем данные
df_outcoming = pd.read_excel(f'{os.getcwd()}/data/drl-data.xlsx', sheet_name='Inventory In')

In [7]:
df_incoming.sample(3)

,#,Product or Service Name,Unit,Quantity/Volume
639877,639878,ELVAN PANDA AG-GARA 45 GR 1X24X6,EDED,24
433180,433181,"PRESIDENT DILIMLI EDAM 150 GR 42,3% 1X10",EDED,70
257109,257110,LAYS PAPRIKA 20 GR 1 x 48,EDED,96


In [8]:
df_outcoming.sample(3)

,#,Product or Service Name,Unit,Quantity/Volume
16511,16512,KETONAL 5% 30G CRM AZ,əd,864
10315,10316,Bağdan Feyxoa Kompotu 1 lt*8,ədəd,240
9804,9805,"Xama ""President"" smetana 20% 200g",əd,240


In [9]:
df_outcoming['unit'] = df_incoming['Unit'].str.lower()
df_incoming['unit'] = df_incoming['Unit'].str.lower()

In [10]:
df_outcoming['unit'].value_counts()

unit
eded       14646
adet.lt     1099
adet.ml      426
ədəd         411
adet.kg      246
adet.cl      119
Name: count, dtype: int64

In [11]:
df_incoming['unit'].value_counts()

unit
eded            800554
adet.lt          61217
ədəd             29627
adet.ml          23627
adet.kg          12295
                 ...  
qab 5kq              1
lt                   1
packa                1
bukum                1
flak (100ml)         1
Name: count, Length: 67, dtype: int64

In [13]:
Counter(" ".join(df_incoming['Product or Service Name']).split()).most_common(50)

[('GR', 631279),
 ('1', 185170),
 ('1X12', 155862),
 ('DAD', 151393),
 ('SEVIMLI', 151228),
 ('PINAR', 101193),
 ('x', 95456),
 ('LAYS', 95451),
 ('1x12', 94284),
 ('200', 83225),
 ('180', 71223),
 ('PRESIDENT', 59237),
 ('LT', 56570),
 ('150', 54808),
 ('KESMIK', 50042),
 ('90', 49971),
 ('QR', 49796),
 ('X', 47754),
 ('500', 47364),
 ('1X24', 44508),
 ('SUXARI', 43497),
 ('XAMA', 42413),
 ('18', 36631),
 ('ML', 35745),
 ('1x1', 34670),
 ('ULKER', 33767),
 ('50', 33451),
 ('KG', 31617),
 ('24', 30583),
 ('1X15', 30069),
 ('PEYNIR', 29382),
 ('TADIM', 29140),
 ('1X8', 28781),
 ('40', 28620),
 ('ELVAN', 28540),
 ('LABNE', 28119),
 ('0%', 27945),
 ('BEBETO', 27920),
 ('KEFIR', 27397),
 ('PENDIR', 27167),
 ('PAPRIKA', 26973),
 ('SUD', 26759),
 ('250', 26645),
 ('YAGI', 26159),
 ('25', 26131),
 ('SUPERFRESH', 24693),
 ('PET', 24627),
 ('MILK', 24502),
 ('DOKTOR', 24145),
 ('20%', 23983)]

In [28]:
flt_chars = [
    ')', '(', ',', '!', '', ':', '|', '>', '<', '[', ']', '...', './', '/\n', '\xa0',
    '«', '»', '_', '?', '~', '=', '\\', ';', '\n', '``', "''", '✅', '/n', '/', '..', '`', "'", '-',
    '✔', '⇒', '"', '$', '%', '&', "'", ',', '^', '`', '{', '}', "('",
]

In [254]:
stop_words = nltk.corpus.stopwords.words('russian') + nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('azerbaijani')
exclude = [x for x in set(stop_words + flt_chars) if len(x) > 0]
exclude.remove('up')
exclude.remove('s')

In [255]:
def process_item(x: str):
    for char in flt_chars:
        x = x.replace(char, ' ').lower()
    item = ' '.join(re.split(r'(\d+)', x))
    while item.count(2 * " ") > 0:
        item = item.replace(2 * " ", " ")
    return ' '.join(st.stem_words([x for x in item.split()]))

In [335]:
def simple_process_item(x: str):
    x = x.lower()
    # for char in flt_chars:
    #     x = x.replace(char, ' ')
    # print(x)
    x = (x.replace('*', ' x ').replace('-', ' ').replace("'", ' ').replace("(", ' ').replace(")", ' ').replace("%", 'pct').replace('"', ' ')
         .replace('<', ' ').replace('>', ' ')).replace(",", '.').replace("/", '.').replace(" s ", 's ')
    item = ' '.join(re.split(r'(\d+)', x))
    while item.count(2 * " ") > 0:
        item = item.replace(2 * " ", " ")
    item = item.replace(' .', '.').replace('. ', '.')
    return ' '.join([x for x in item.split() if x not in exclude])

In [336]:
exclude

['of',
 'istifadə',
 'harada',
 'да',
 'further',
 'перед',
 'məhz',
 'раз',
 "shouldn't",
 'был',
 'niyə',
 'тоже',
 "you'd",
 'my',
 ')',
 "''",
 '-',
 "couldn't",
 'it',
 "wasn't",
 'own',
 'about',
 'после',
 'когда',
 'arasında',
 'an',
 'ли',
 '(',
 'əslində',
 'yəni',
 '✅',
 'более',
 'мы',
 'об',
 'why',
 'много',
 'ay',
 'них',
 'bu',
 "mustn't",
 'какой',
 'yalnız',
 'birşey',
 'being',
 'here',
 'də',
 'быть',
 'very',
 'təəssüf',
 "haven't",
 'nə',
 'düz',
 'меня',
 'про',
 'sadəcə',
 'от',
 "'",
 'bəy',
 'onu',
 'ha',
 'мой',
 '✔',
 'siz',
 'mənə',
 ',',
 '&',
 "didn't",
 'потом',
 'few',
 'ondan',
 'больше',
 'сам',
 'а',
 'i',
 'your',
 'yourself',
 'artıq',
 'while',
 'no',
 'он',
 'как',
 'couldn',
 'в',
 'ə',
 'd',
 'indi',
 'mirşey',
 "aren't",
 'onun',
 'можно',
 'yourselves',
 'ни',
 'этом',
 'amma',
 'which',
 'ей',
 '⇒',
 'два',
 'weren',
 'between',
 'again',
 'çox',
 "isn't",
 'но',
 'isə',
 "shan't",
 'had',
 'sənin',
 'я',
 "hasn't",
 'üç',
 'за',
 'bilər',
 

In [337]:
# exclude

In [338]:
text = df_outcoming.sample(1)['Product or Service Name'].values[0]
print(text)
simple_process_item(text)

Sev 5 Kq Burgu


'sev 5 kq burgu'

In [339]:
text = df_incoming.sample(1)['Product or Service Name'].values[0]
print(text)
simple_process_item(text)

KUZYA KUKURUZ 85 GR 1X16


'kuzya kukuruz 85 gr 1 x 16'

In [340]:
df_outcoming['items'] = df_outcoming['Product or Service Name'].apply(lambda x: simple_process_item(x))
df_incoming['items'] = df_incoming['Product or Service Name'].apply(lambda x: simple_process_item(x))

In [341]:
Counter(" ".join(df_outcoming['items']).split()).most_common(150)

[('x', 10399),
 ('qr', 4951),
 ('pct', 4903),
 ('milla', 2809),
 ('1', 2571),
 ('12', 2282),
 ('200', 1679),
 ('lays', 1570),
 ('ml', 1557),
 ('24', 1504),
 ('lt', 1497),
 ('dərman', 1449),
 ('yoqurt', 1408),
 ('48', 1385),
 ('6', 1238),
 ('100', 1192),
 ('20', 1185),
 ('president', 1168),
 ('g', 1120),
 ('kq', 1099),
 ('№', 1017),
 ('150', 1005),
 ('3.5', 1000),
 ('dsp', 944),
 ('25', 923),
 ('gr', 909),
 ('pendir', 848),
 ('xama', 823),
 ('18', 792),
 ('süd', 770),
 ('90', 731),
 ('30', 715),
 ('xrustim', 692),
 ('10', 661),
 ('300', 645),
 ('l', 637),
 ('q', 622),
 ('50', 619),
 ('suxari', 618),
 ('1.5', 612),
 ('8', 596),
 ('0.5', 593),
 ('5', 554),
 ('ayran', 551),
 ('paprika', 525),
 ('şaftalı', 506),
 ('sevimli', 500),
 ('dad', 500),
 ('qaymaq', 466),
 ('3', 463),
 ('27', 462),
 ('kəsmik', 435),
 ('2', 421),
 ('snq', 418),
 ('мг', 408),
 ('40', 397),
 ('salmonella', 396),
 ('qatıq', 393),
 ('kefir', 391),
 ('16', 386),
 ('mimi', 380),
 ('мл', 378),
 ('doritos', 376),
 ('4', 373)

In [342]:
Counter(" ".join(df_incoming['items']).split()).most_common(150)

[('1', 1013891),
 ('x', 950016),
 ('gr', 699170),
 ('12', 276891),
 ('pct', 206969),
 ('dad', 151392),
 ('sevimli', 151228),
 ('24', 114888),
 ('pinar', 101193),
 ('lays', 95451),
 ('200', 93967),
 ('6', 74396),
 ('180', 72909),
 ('20', 72431),
 ('lt', 61267),
 ('president', 59237),
 ('150', 55888),
 ('qr', 54496),
 ('90', 50787),
 ('kesmik', 50042),
 ('500', 49153),
 ('30', 48557),
 ('15', 47800),
 ('18', 47650),
 ('suxari', 43497),
 ('10', 43147),
 ('25', 42919),
 ('xama', 42353),
 ('ml', 39934),
 ('kg', 36846),
 ('8', 36076),
 ('40', 35656),
 ('50', 35228),
 ('ulker', 33767),
 ('16', 32089),
 ('250', 29926),
 ('peynir', 29382),
 ('tadim', 29140),
 ('0', 28819),
 ('elvan', 28540),
 ('60', 28309),
 ('labne', 28119),
 ('bebeto', 27920),
 ('9', 27676),
 ('kefir', 27397),
 ('pendir', 27169),
 ('paprika', 26973),
 ('sud', 26818),
 ('yagi', 26169),
 ('0.5', 26066),
 ('2.5', 25530),
 ('4', 25485),
 ('300', 25248),
 ('27', 25122),
 ('pet', 24731),
 ('superfresh', 24693),
 ('milk', 24502),
 (

In [343]:
def count_digit_share(x: str):
    counter = 0
    for char in x:
        if char.isdigit():
            counter += 1
    return counter/len(x.replace(' ', ''))

In [344]:
count_digit_share('7 up 1 lt x 12')

0.4444444444444444

In [345]:
def count_common_words(str1, str2):
    set1 = set([x for x in str1.lower().split() if len(x) > 3 and not x.isdigit()])
    set2 = set([x for x in str2.lower().split() if len(x) > 3 and not x.isdigit()])
    # print(set1.intersection(set2))
    # return len(set1.intersection(set2)), (len(set1) + len(set2))/2, len(set1.intersection(set2))/((len(set1) + len(set2))/2)
    if ((len(set1) + len(set2))/2) == 0:
        return 0
    return len(set1.intersection(set2))/((len(set1) + len(set2))/2)

In [346]:
def count_common_digits(str1, str2):
    set1 = set([x for x in str1.lower().split() if x.isdigit()])
    set2 = set([x for x in str2.lower().split() if x.isdigit()])
    # print(set1.intersection(set2))
    # return len(set1.intersection(set2)), (len(set1) + len(set2))/2, len(set1.intersection(set2))/((len(set1) + len(set2))/2)
    if ((len(set1) + len(set2))/2) == 0:
        return 0
    return len(set1.intersection(set2))/((len(set1) + len(set2))/2)

In [347]:
subquery = df_incoming.sample(1)['items'].values[0]
print(subquery)
answers = proc.extract(
                subquery,
                set(df_outcoming['items'].to_list()),
                scorer=fuzz.token_set_ratio,
                limit=1,
                # score_cutoff=75
)
[print(x) for x in answers]
if len(answers) > 0:
    print(count_common_words(subquery, answers[0][0]))
    print(count_common_digits(subquery, answers[0][0]))

doktor milk kesmik 0 pct 180 qr uzumlu 1 x 1
('sevimli dad kesmik kaunas 0 pct 180 gr 1 x 1', 66.66666666666666, 989)
0.2857142857142857
1.0


In [348]:
df_outcoming[df_outcoming['items'] == '7 up 1 lt x 12']

,#,Product or Service Name,Unit,Quantity/Volume,unit,items
4930,4931,7up 1 lt*12,ƏDƏD,1800,eded,7 up 1 lt x 12
4944,4945,7up 1 lt*12,ƏDƏD,1800,adet.kg,7 up 1 lt x 12
6614,6615,7up 1 lt*12,ƏDƏD,900,eded,7 up 1 lt x 12
6627,6628,7up 1 lt*12,ƏDƏD,900,eded,7 up 1 lt x 12
6632,6633,7up 1 lt*12,ƏDƏD,900,eded,7 up 1 lt x 12
6637,6638,7up 1 lt*12,ƏDƏD,900,eded,7 up 1 lt x 12
6649,6650,7up 1 lt*12,ƏDƏD,900,eded,7 up 1 lt x 12
6679,6680,7up 1 lt*12,ƏDƏD,900,adet.ml,7 up 1 lt x 12
6698,6699,7up 1 lt*12,ƏDƏD,900,eded,7 up 1 lt x 12


In [595]:
%%time
# Обрабатываем все товары из внешних источников и находим для них пару из предобработанного стобца. Score - степень схожести
to_find = df_incoming['items'].to_list()
candidates = set(df_outcoming['items'].to_list())

res_ext, res_int, score, score_w, score_d = [], [], [], [], []
for item in tqdm(to_find[:]):
    res = proc.extract(
                item,
                candidates,
                scorer=fuzz.token_set_ratio,
                limit=1
    )[0]
    res_ext.append(item)
    res_int.append(res[0])
    score.append(res[1])
    if len(res) > 0:
        score_w.append(count_common_words(item, res[0]))
        score_d.append(count_common_digits(item, res[0]))
        

100%|██████████| 937415/937415 [1:03:30<00:00, 246.01it/s]

CPU times: user 1h 3min 24s, sys: 7.74 s, total: 1h 3min 32s
Wall time: 1h 3min 30s


In [596]:
from fastparquet import ParquetFile, write as parq_write

In [597]:
df_outcoming.shape, df_incoming.shape

((16947, 6), (937415, 6))

In [598]:
comp = pd.DataFrame()

In [599]:
comp['right'] = res_ext

In [600]:
comp['left'] = res_int

In [601]:
comp['score'] = score
comp['score_w'] = score_w
comp['score_d'] = score_d

In [602]:
parq_write(
    f'{os.getcwd()}/comp.parq',
    comp,
    compression='GZIP'
)

In [603]:
comp.shape

(937415, 5)

In [604]:
comp[(comp.score > 75) & (comp.score > 0.4) & (comp.score_d > 0.4)].shape

(371964, 5)

In [605]:
comp[(comp.score > 75) & (comp.score > 0.4) & (comp.score_d > 0.4)].sample(10)

,right,left,score,score_w,score_d
361115,doritos barbecue bbq 72 gr 1 x 26,doritos bbq 72 g x 26,95.000000,0.666667,0.800000
647308,nesquik snack latte 26 gr 1 x 20,nesquik snack latte 26 qr x 20,94.736842,1.000000,0.800000
800744,sochnaya dolina alma 0.95 lt 1 x 12,7 up 1 lt x 12,78.260870,0.000000,0.800000
367037,lays sir 150 gr 1 x 18,lays pendir 150 qr x 18,75.555556,0.666667,0.800000
921729,lays sir 150 gr 1 x 18,lays pendir 150 qr x 18,75.555556,0.666667,0.800000
826766,sevimli dad gatig 4.5 pct 450 gr 1 x 22,sevimli dad gatig 1.5 pct 450 gr 1 x 22,97.435897,1.000000,1.000000
503351,lays baharat 150 gr 1 x 18,lays dalğavari baharat 150 qr x 18,89.361702,0.800000,0.800000
253116,sevimli dad urekli pendir ceki 1 x 3,sevimli dad rossiyskiy pendir 1 x 1,80.000000,0.571429,0.666667
518807,sonti multivitamin 200 gr 1 x 27,jalə multivitamin p 200 ml x 27,80.769231,0.500000,0.800000
515677,sevimli dad kesmik ciyelekli 0 pct 180 gr 1 x 1,sevimli dad kesmik ciyelekli 0 pct 180 gr 1 x 1,100.000000,1.000000,1.000000


In [606]:
comp.head()

,right,left,score,score_w,score_d
0,lays shashlik 90 gr 1 x 25,lays shashlyk 150 qr x 18,78.431373,0.5,0.000000
1,burcu ketcup aci bufe tipi 650 gr 1 x 12,7 up 1 lt x 12,60.000000,0.0,0.666667
2,burcu domates tomati 320 gr 1 x 12,natura tomat prizma 1 lt x 12,60.317460,0.0,0.800000
3,qlavprodukt gobelek opyata 580 qr 1 x 12,7 up 1 lt x 12,60.000000,0.0,0.666667
4,pamuk lokum findiqli portagalli 450 gr 1 x 12,7 up 1 lt x 12,60.000000,0.0,0.666667


In [607]:
sub_result = pd.concat(
    [
        df_incoming,
        comp
    ],
    axis=1,
)

In [608]:
sub_result.head()

,#,Product or Service Name,Unit,Quantity/Volume,unit,items,right,left,score,score_w,score_d
0,1,LAYS SHASHLIK 90 GR 1 x 25,EDED,25,eded,lays shashlik 90 gr 1 x 25,lays shashlik 90 gr 1 x 25,lays shashlyk 150 qr x 18,78.431373,0.5,0.000000
1,2,BURCU KETCUP ACI BUFE TIPI 650 GR 1X12,EDED,12,eded,burcu ketcup aci bufe tipi 650 gr 1 x 12,burcu ketcup aci bufe tipi 650 gr 1 x 12,7 up 1 lt x 12,60.000000,0.0,0.666667
2,3,BURCU DOMATES TOMATI 320GR 1X12,EDED,5,eded,burcu domates tomati 320 gr 1 x 12,burcu domates tomati 320 gr 1 x 12,natura tomat prizma 1 lt x 12,60.317460,0.0,0.800000
3,4,QLAVPRODUKT GOBELEK OPYATA 580 QR 1X12,EDED,12,eded,qlavprodukt gobelek opyata 580 qr 1 x 12,qlavprodukt gobelek opyata 580 qr 1 x 12,7 up 1 lt x 12,60.000000,0.0,0.666667
4,5,PAMUK LOKUM FINDIQLI PORTAGALLI 450 GR 1x12,EDED,6,eded,pamuk lokum findiqli portagalli 450 gr 1 x 12,pamuk lokum findiqli portagalli 450 gr 1 x 12,7 up 1 lt x 12,60.000000,0.0,0.666667


In [609]:
sub_result.shape

(937415, 11)

In [678]:
df_outcoming.head(3)

,#,Product or Service Name,Unit,Quantity/Volume,unit,items
0,1,Mis boru VRV 9.52x0.80,kq,"9,85",eded,mis boru vrv 9.52 x 0.80
1,2,Mis boru VRV 16x1,kq,"8,65",eded,mis boru vrv 16 x 1
2,3,"Bri ""President"" 3.4kg 60% folga Yumuşaq Pendir",kg,"76,82",eded,bri president 3.4 kg 60 pct folga yumuşaq pendir


In [679]:
df_incoming.head(3)

,#,Product or Service Name,Unit,Quantity/Volume,unit,items
0,1,LAYS SHASHLIK 90 GR 1 x 25,EDED,25,eded,lays shashlik 90 gr 1 x 25
1,2,BURCU KETCUP ACI BUFE TIPI 650 GR 1X12,EDED,12,eded,burcu ketcup aci bufe tipi 650 gr 1 x 12
2,3,BURCU DOMATES TOMATI 320GR 1X12,EDED,5,eded,burcu domates tomati 320 gr 1 x 12


In [610]:
comp_int = sub_result.merge(df_outcoming, how='left', left_on='left', right_on='items')

In [611]:
comp_int.head()

,#_x,Product or Service Name_x,Unit_x,Quantity/Volume_x,unit_x,items_x,right,left,score,score_w,score_d,#_y,Product or Service Name_y,Unit_y,Quantity/Volume_y,unit_y,items_y
0,1,LAYS SHASHLIK 90 GR 1 x 25,EDED,25,eded,lays shashlik 90 gr 1 x 25,lays shashlik 90 gr 1 x 25,lays shashlyk 150 qr x 18,78.431373,0.5,0.0,753,Lay's Shashlyk 150qr X18,əd,24516,adet.kg,lays shashlyk 150 qr x 18
1,1,LAYS SHASHLIK 90 GR 1 x 25,EDED,25,eded,lays shashlik 90 gr 1 x 25,lays shashlik 90 gr 1 x 25,lays shashlyk 150 qr x 18,78.431373,0.5,0.0,1188,Lay's Shashlyk 150qr X18,əd,18144,eded,lays shashlyk 150 qr x 18
2,1,LAYS SHASHLIK 90 GR 1 x 25,EDED,25,eded,lays shashlik 90 gr 1 x 25,lays shashlik 90 gr 1 x 25,lays shashlyk 150 qr x 18,78.431373,0.5,0.0,1192,Lay's Shashlyk 150qr X18,əd,18144,eded,lays shashlyk 150 qr x 18
3,1,LAYS SHASHLIK 90 GR 1 x 25,EDED,25,eded,lays shashlik 90 gr 1 x 25,lays shashlik 90 gr 1 x 25,lays shashlyk 150 qr x 18,78.431373,0.5,0.0,1226,Lay's Shashlyk 150qr X18,əd,17280,eded,lays shashlyk 150 qr x 18
4,1,LAYS SHASHLIK 90 GR 1 x 25,EDED,25,eded,lays shashlik 90 gr 1 x 25,lays shashlik 90 gr 1 x 25,lays shashlyk 150 qr x 18,78.431373,0.5,0.0,1500,Lay's Shashlyk 150qr X18,əd,13824,eded,lays shashlyk 150 qr x 18


In [612]:
comp_int.shape

(20996925, 17)

In [613]:
comp_int = comp_int.drop_duplicates(subset=['#_x']) 
comp_int.shape

(937415, 17)

In [614]:
# x - incoming y - outcoming
comp_int.sample(5)

,#_x,Product or Service Name_x,Unit_x,Quantity/Volume_x,unit_x,items_x,right,left,score,score_w,score_d,#_y,Product or Service Name_y,Unit_y,Quantity/Volume_y,unit_y,items_y
20193412,896318,3 JELANIYA MAYONEZ PROVANSAL 67 % 190 GR 1x60,EDED,12,eded,3 jelaniya mayonez provansal 67 pct 190 gr 1 x 60,3 jelaniya mayonez provansal 67 pct 190 gr 1 x 60,sevimli dad xama kaunas 10 pct 200 gr 1 x 12,55.913978,0.000000,0.222222,1961,SEVIMLI DAD XAMA KAUNAS 10% 200 GR 1X12,ED,10003,eded,sevimli dad xama kaunas 10 pct 200 gr 1 x 12
3801507,169283,LAYS KLASIK 90 GR 1x12,EDED,72,eded,lays klasik 90 gr 1 x 12,lays klasik 90 gr 1 x 12,lays sol dsp 90 qr x 12,72.340426,0.666667,0.800000,495,Lay's Sol DSP 90qr X12,əd,33408,eded,lays sol dsp 90 qr x 12
11751348,520094,LAYS HEDIYYELI QUTU 1X1,EDED,5,eded,lays hediyyeli qutu 1 x 1,lays hediyyeli qutu 1 x 1,lays pendir 150 qr x 18,60.869565,0.400000,0.000000,719,Lay's Pendir 150 qr X18,əd,25056,eded,lays pendir 150 qr x 18
19459744,864101,ULKER PUDING ICIM SOKOLADLI 60 GR 1X24,EDED,24,eded,ulker puding icim sokoladli 60 gr 1 x 24,ulker puding icim sokoladli 60 gr 1 x 24,puding dolcia shokoladlı 100 gr 4 x 6,75.324675,0.285714,0.000000,3557,"Puding ""Dolcia"" Shokoladlı 100GR 4X6",əd,3600,eded,puding dolcia shokoladlı 100 gr 4 x 6
16534554,731758,MIRINDA ALMA 1 LT 1X12,ADET.LT,12,adet.lt,mirinda alma 1 lt 1 x 12,mirinda alma 1 lt 1 x 12,mirinda alma 1 lt x 12,100.000000,1.000000,1.000000,2195,Mirinda Alma 1 lt*12,ƏDƏD,8640,eded,mirinda alma 1 lt x 12


In [615]:
comp_int['unit_x'].value_counts()

unit_x
eded            800554
adet.lt          61217
ədəd             29627
adet.ml          23627
adet.kg          12295
                 ...  
qab 5kq              1
lt                   1
packa                1
bukum                1
flak (100ml)         1
Name: count, Length: 67, dtype: int64

In [616]:
comp_int['unit_y'].value_counts()

unit_y
eded       830019
adet.lt     48471
adet.ml     33587
adet.kg     19383
ədəd         3044
adet.cl      2911
Name: count, dtype: int64

In [617]:
comp_int.columns

Index(['#_x', 'Product or Service Name_x', 'Unit_x', 'Quantity/Volume_x',
       'unit_x', 'items_x', 'right', 'left', 'score', 'score_w', 'score_d',
       '#_y', 'Product or Service Name_y', 'Unit_y', 'Quantity/Volume_y',
       'unit_y', 'items_y'],
      dtype='object')

In [618]:
comp_int = comp_int[[
          'items_x', 'items_y',
          'Quantity/Volume_x', 'unit_x', 'Quantity/Volume_y', 'unit_y',
          'score', 'score_w', 'score_d',
          'Product or Service Name_x', 'Product or Service Name_y', ]]

In [619]:
comp_int_true = comp_int[(comp_int.score > 75) & (comp_int.score_w > 0.4) & (comp_int.score_d > 0.4)]

In [620]:
comp_int_doubt = comp_int.drop(comp_int_true.index, axis=0)

In [659]:
comp_int_doubt['items_y']  = ''
comp_int_doubt['Quantity/Volume_y'] = 0
comp_int_doubt['unit_y'] = ''
comp_int_doubt['Product or Service Name_y'] = ''

In [660]:
comp_scored = pd.concat([comp_int_true, comp_int_doubt])

In [661]:
comp_scored.shape

(937415, 18)

In [662]:
comp_scored.sample(10)

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score,score_w,score_d,Product or Service Name_x,Product or Service Name_y,unit_common,unit_common_set,unit_2pc_x,unit_2pc_y,unit_5pc_x,unit_5pc_y,unit_common_str
6526292,sevimli dad kesmik yagli 9 pct 180 gr 1 x 1,sevimli dad kesmik yagsiz 0.3 pct 180 gr 1 x 1,4.0,eded,4744.0,eded,91.764706,0.666667,0.800000,SEVIMLI DAD KESMIK YAGLI 9% 180 GR 1x1,SEVIMLI DAD KESMIK YAGSIZ 0.3% 180 GR 1x1,eded eded,{eded},NaN,NaN,True,True,eded
7613107,pinar sut organik 500 ml 1 x 12,,12,adet.ml,0.0,,63.157895,0.000000,0.400000,PINAR SUT ORGANIK 500 ML 1x12,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2358347,pinar peynir labne 2750 gr 1 x 4,,1,eded,0.0,,70.769231,0.333333,0.000000,PINAR PEYNIR LABNE 2750 GR 1x4,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8303968,lays krab 90 gr 1 x 25,lays krab 90 q dsp snq ftb x 25,6.0,eded,22800.0,eded,87.179487,1.000000,0.800000,LAYS KRAB 90 GR 1 x 25,Lay's Krab 90q DSP SNQ Ftb X25,eded eded,{eded},NaN,NaN,True,True,eded
11280567,hugo reitzel tursu salatalik 720 ml tatli 1 x 12,,9,adet.ml,0.0,,60.000000,0.000000,0.666667,HUGO REITZEL TURSU SALATALIK 720 ML (TATLI) 1X12,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1369377,sevimli dad dovga 2.5 pct 220 gr 1 x 25,,80,eded,0.0,,84.615385,0.500000,0.333333,SEVIMLI DAD DOVGA 2.5% 220 GR 1X25,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1107879,ulker puding cokokrem findig 100 gr 1 x 24,,1,eded,0.0,,71.264368,0.500000,0.800000,ULKER PUDING COKOKREM FINDIG 100 GR 1X24,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11981666,burcu ketcup tatli 250 gr 1 x 12,,4,eded,0.0,,60.000000,0.000000,0.666667,BURCU KETCUP TATLI 250 GR 1X12,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9275806,pamuk lokum findiqli portagalli 250 gr 1 x 12,,3,eded,0.0,,60.000000,0.000000,0.666667,PAMUK LOKUM FINDIQLI PORTAGALLI 250 GR 1x12,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19157521,berrak aci sus biberi tursusu 370 ml 1 x 12,,6,adet.ml,0.0,,60.000000,0.000000,0.666667,BERRAK ACI SUS BIBERI TURSUSU 370 ML 1X12,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [663]:
comp_int.shape, comp_int_true.shape, comp_int_doubt.shape

((937415, 11), (299264, 18), (638151, 11))

In [664]:
comp_int_true.sample(5)

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score,score_w,score_d,Product or Service Name_x,Product or Service Name_y,unit_common,unit_common_set,unit_2pc_x,unit_2pc_y,unit_5pc_x,unit_5pc_y,unit_common_str
20898405,suxari smetan 30 gr 1 x 40,suxari xrustim smetan 30 qr x 40,40.0,eded,57600.0,adet.ml,89.361702,0.800000,0.800000,SUXARI SMETAN 30 GR 1 X 40,Suxari Xrustim Smetan 30 qr X40,eded adet.ml,"{eded, ml, adet}",NaN,ml,True,True,eded ml adet
17254328,sevimli dad xama kaunasskaya 20 pct 350 gr 1 x 12,sevimli dad xama kaunasskaya 20 pct 350 gr 1 x 12,12.0,eded,5094.0,eded,100.000000,1.000000,1.000000,SEVIMLI DAD XAMA KAUNASSKAYA 20% 350 GR 1X12,SEVIMLI DAD XAMA KAUNASSKAYA 20% 350 GR 1X12,eded eded,{eded},NaN,NaN,True,True,eded
18733367,sevimli dad kesmik kaunas 0.3 pct 180 gr 1 x 1,sevimli dad kesmik kaunas 0 pct 180 gr 1 x 1,3.0,eded,8318.0,eded,97.674419,1.000000,0.800000,SEVIMLI DAD KESMIK KAUNAS 0.3% 180 GR1x1,SEVIMLI DAD KESMIK KAUNAS 0% 180 GR1x1,eded eded,{eded},NaN,NaN,True,True,eded
20118563,suxari biftekli 130 gr 1 x 24,suxari xrustim biftekli 130 qr x 24,5.0,eded,23328.0,eded,90.566038,0.800000,0.800000,SUXARI BIFTEKLI 130 GR 1 X 24,Suxari Xrustim Biftekli 130qr X24,eded eded,{eded},NaN,NaN,True,True,eded
14801377,sevimli dad kesmik gaysili 9 pct 180 gr 1 x 1,sevimli dad kesmik albalili 0 pct 180 gr 1 x 1,24.0,eded,6900.0,adet.ml,89.655172,0.666667,0.666667,SEVIMLI DAD KESMIK GAYSILI 9% 180 GR 1x1,SEVIMLI DAD KESMIK ALBALILI 0% 180 GR1x1,eded adet.ml,"{eded, ml, adet}",NaN,ml,True,True,eded ml adet


In [665]:
import numpy as np

In [666]:
def extract_pieces_two_char(x: str):
    for pc in str(x).replace('.', ' ').split():
        if len(pc) == 2:
            return pc
    return np.nan

In [667]:
def extract_pieces_five_char(x: str):
    for pc in str(x).replace('.', ' ').split():
        if len(pc) == 4:
            return True
    return np.nan

In [668]:
comp_scored['unit_common'] = comp_scored['unit_x'] + ' ' +  comp_int_true['unit_y']

In [669]:
comp_scored['unit_common_set'] = comp_scored['unit_common'].apply(lambda x: set(str(x).replace('.', ' ').split()))

In [670]:
comp_scored['unit_2pc_x'] = comp_scored['unit_x'].apply(lambda x: extract_pieces_two_char(x))
comp_scored['unit_2pc_y'] = comp_scored['unit_y'].apply(lambda x: extract_pieces_two_char(x))

In [671]:
comp_scored['unit_common_str'] = comp_scored['unit_common_set'].apply(lambda x: str(list(x)[0]) if len(list(x)) == 0 else ' '.join(list(x)))

In [672]:
# x - incoming y - outcoming
comp_scored.sample(10)

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score,score_w,score_d,Product or Service Name_x,Product or Service Name_y,unit_common,unit_common_set,unit_2pc_x,unit_2pc_y,unit_5pc_x,unit_5pc_y,unit_common_str
18087427,pinar peynir labne 400 gr 1 x 12,,8,eded,0.0,,73.846154,0.333333,0.00,PINAR PEYNIR LABNE 400 GR 1x12,,NaN,{nan},NaN,NaN,NaN,NaN,nan
18305769,sevimli dad kesmik yagli 9 pct 180 gr 1 x 1,sevimli dad kesmik yagsiz 0.3 pct 180 gr 1 x 1,4.0,eded,4744.0,eded,91.764706,0.666667,0.80,SEVIMLI DAD KESMIK YAGLI 9% 180 GR 1x1,SEVIMLI DAD KESMIK YAGSIZ 0.3% 180 GR 1x1,eded eded,{eded},NaN,NaN,True,True,eded
9844778,mimi sud ciyelekli 200 ml 1 x 24,mimi süd çiyələkli 200 ml x 24,6.0,eded,1200.0,eded,77.419355,0.500000,0.80,MIMI SUD CIYELEKLI 200ML 1X24,Mimi Süd Çiyələkli 200ml *24,eded eded,{eded},NaN,NaN,True,True,eded
19025856,sevimli dad xama 10 pct 200 gr 1 x 20,sevimli dad xama kaunas 10 pct 200 gr 1 x 12,15.0,eded,10003.0,eded,95.774648,0.800000,0.75,SEVIMLI DAD XAMA 10% 200 GR 1X20,SEVIMLI DAD XAMA KAUNAS 10% 200 GR 1X12,eded eded,{eded},NaN,NaN,True,True,eded
7721026,lays paprika 90 gr 1 x 25 p,lays paprika 90 q x 25,25.0,eded,34800.0,eded,95.238095,1.000000,0.80,LAYS PAPRIKA 90 GR 1 x 25 P,Lay's Paprika 90q X25,eded eded,{eded},NaN,NaN,True,True,eded
13428698,doktor milk xama 15 pct 350 qr 1 x 12,,4,eded,0.0,,72.727273,0.400000,0.00,DOKTOR MILK XAMA 15% 350 QR 1x12,,NaN,{nan},NaN,NaN,NaN,NaN,nan
2628722,sev makaron tuk 1 kg 1 x 10,,20,eded,0.0,,61.111111,0.500000,0.00,SEV MAKARON TUK 1 KG 1X10,,NaN,{nan},NaN,NaN,NaN,NaN,nan
4646763,suxari biftekli 30 gr 1 x 40,suxari xrustim biftekli 30 qr sng l x 40,40.0,eded,72000.0,eded,90.196078,0.800000,0.80,SUXARI BIFTEKLI 30 GR 1 X 40,Suxari Xrustim Biftekli 30 qr SNG l X40,eded eded,{eded},NaN,NaN,True,True,eded
8045655,deste siyah zeytin konfit 250 gr vkm 300 360 1...,,5,eded,0.0,,57.777778,0.000000,0.25,DESTE SIYAH ZEYTIN KONFIT 250 GR VKM 300-360 1x20,,NaN,{nan},NaN,NaN,NaN,NaN,nan
16560311,suxari kalmar 90 gr 1 x 24,suxari xrustim kalmar 90 qr dspl x 24,24.0,eded,19008.0,eded,89.361702,0.666667,0.80,SUXARI KALMAR 90 GR 1 X 24,Suxari Xrustim Kalmar 90 qr DSPl X24,eded eded,{eded},NaN,NaN,True,True,eded


In [673]:
comp_scored['quant_in'] = comp_scored['Quantity/Volume_x'].astype('str').str.replace(',', '.').fillna(0).astype('float')
comp_scored['quant_out'] = comp_scored['Quantity/Volume_y'].astype('str').str.replace(',', '.').fillna(0).astype('float')

In [677]:
comp_scored.groupby(by=['items_x', 'unit_common_str']).agg({'quant_in':'sum','quant_out':'sum'}).sort_values('unit_common_str')

,,quant_in,quant_out
items_x,unit_common_str,,
president xama 15 pct 350 gr,cl adet ədəd,1634.0,31536.0
delverde spagetti bio x 4 500 gr 1 x 24,eded,1934.0,1920000.0
sev makaron spagetti 5 kg 1 x 1,eded,7420.0,709800.0
president kesme chedar 40 pct 150 gr 1 x 15,eded,44922.0,24560250.0
suxari xolodeçs xrenom 90 gr 1 x 24,eded,15840.0,15909696.0
...,...,...,...
linex n 16 08.2021 357,ədəd,127.0,8925.0
nakom 250 + 25 mg № 100 09.2022,ədəd,22.0,4320.0
esomeprazol 20 mg n 28 08.2021,ədəd,1.0,100.0


In [680]:
comp_scored.shape

(937415, 20)

In [766]:
def remove_numbers(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    # while text.count(2 * " ") > 0:
    #     text = text.replace(2 * " ", " ")
    return ' '.join([x for x in text.split() if len(x) > 2 and x not in ('pct', 'dad')][:3])

In [767]:
comp_scored['items_x_numberless'] = comp_scored['items_x'].apply(lambda x: remove_numbers(x))

In [768]:
comp_scored[['items_x_numberless', 'items_x']].sample(5)

,items_x_numberless,items_x
2900182,pinar sucuk klasik,pinar sucuk klasik kangal 225 gr 1 x 10
20635106,berrak aci sus,berrak aci sus biberi tursusu 370 ml 1 x 12
17424369,elvan today roll,elvan today roll findiq rulet keks 100 gr 1 x ...
6679649,lays ridged paprika,lays ridged paprika 90 gr 1 x 12
8014083,president kere yagi,president kere yagi 82 pct 200 gr 1 x 40


In [770]:
Counter(" ".join(comp_scored['items_x_numberless']).split()).most_common(150)

[('sevimli', 151228),
 ('pinar', 101193),
 ('lays', 95451),
 ('president', 59237),
 ('kesmik', 50042),
 ('suxari', 43497),
 ('xama', 42353),
 ('ulker', 33767),
 ('tadim', 29140),
 ('peynir', 28892),
 ('elvan', 28540),
 ('labne', 28119),
 ('bebeto', 27920),
 ('kefir', 27397),
 ('paprika', 26973),
 ('pendir', 24770),
 ('superfresh', 24693),
 ('yagi', 24473),
 ('milk', 24335),
 ('doktor', 24145),
 ('doritos', 23300),
 ('sud', 23240),
 ('gatig', 22757),
 ('ridged', 21678),
 ('qlavprodukt', 21570),
 ('puding', 21407),
 ('klasik', 20091),
 ('kaunasskaya', 19690),
 ('sir', 19285),
 ('pendiri', 19222),
 ('today', 17963),
 ('yagli', 16512),
 ('sut', 15250),
 ('kaunas', 15095),
 ('icim', 14852),
 ('kesme', 14638),
 ('kirlangic', 14106),
 ('biftekli', 12854),
 ('acili', 12611),
 ('cheetos', 12300),
 ('sochnaya', 12277),
 ('canax', 12096),
 ('misir', 11872),
 ('kere', 10950),
 ('burcu', 10929),
 ('bagdat', 10767),
 ('shashlik', 10700),
 ('berrak', 10485),
 ('milla', 10194),
 ('asci', 10164),
 ('sa

In [705]:
import re
import time
from sentence_transformers import SentenceTransformer, util

/home/varsey/PycharmProjects/drl-case/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [771]:
def clusterize(model: object, job_name_list: list, **kwargs) -> list:
    """Клсастеризация набора должностей."""
    # Apply encoding.
    print("Encode the corpus. This might take a while")
    corpus_embeddings = model.encode(
        job_name_list, 
        batch_size=32, 
        show_progress_bar=True, 
        convert_to_tensor=True
    )
    # Get clusters
    print("Start clustering")
    start_time = time.time()
    #Two parameters to tune:
    #min_cluster_size: Only consider cluster that have at least 25 elements
    #threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
    clusters = util.community_detection(corpus_embeddings, **kwargs)
    print("Clustering done after {:.2f} sec".format(time.time() - start_time))
    return clusters

In [ ]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [772]:
len(set(comp_scored['items_x_numberless'].to_list())), len(comp_scored['items_x_numberless'].to_list())

(2910, 937415)

In [773]:
job_name_list = list(set(comp_scored['items_x_numberless'].to_list()))
cluster = clusterize(model, job_name_list)

Encode the corpus. This might take a while


Batches:   0%|          | 0/91 [00:00<?, ?it/s]

Start clustering
Clustering done after 0.04 sec


In [774]:
import random

In [775]:
for cl in cluster[:]:
    for member in random.sample(cl, 5):
        print(comp_scored['items_x_numberless'].to_list()[member])
    print(len(cl))
    print()
    

doritos acili
cheetos biftekli kitir
sevimli kefir
sevimli kesmik gaysili
sevimli xama
233

cheetos biftekli kitir
president kesme chedar
lays rebrishki qril
cheetos fistikli
sevimli xama kaunas
94

ulker pendir labne
sevimli kefir
president xama
ulker puding alpella
doritos taco
79

sevimli kefir kaunas
sevimli kere yag
ulker pendir labne
suxari smetan
president kefir
68

sevimli kefir
suxari biftekli
suxari shashlik
sandora multi
sevimli rossiyskiy pendir
62

sevimli kefir
sevimli kesmik kaunas
milla ayran
sonti multivitamin
sevimli gatig
51

president yagli
sevimli xama kaunasskaya
ulker puding alpella
sevimli gatig
sevimli kesmik gaysili
41

parmalat sud uht
sevimli kefir
lays baharat
sevimli dovga
ulker pendir labne
36

lays klasik
sevimli kesmik uzumlu
president gobelek
lays paprika
milla yoqurt shaftali
33

ulker puding alpella
mimi sud bananli
sevimli gatig
ulker pendir labne
sevimli kesmik uzumlu
30

ulker pendir labne
lays sir
lays paprika
sevimli xama
lays krab
30

ulker pen

In [734]:
[len(x) for x in cluster]

[215,
 86,
 69,
 66,
 59,
 58,
 48,
 41,
 39,
 39,
 37,
 34,
 34,
 30,
 28,
 27,
 23,
 23,
 22,
 21,
 21,
 20,
 19,
 18,
 17,
 17,
 17,
 16,
 15,
 15,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 13,
 13,
 13,
 13,
 13,
 13,
 12,
 12,
 12,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10]

In [776]:
import spacy

# !python -m spacy download en_core_web_sm xx_ent_wiki_sm
nlp = spacy.load("en_core_web_sm")
for member in random.sample(cluster[:1][0], 10):
    doc = nlp(comp_scored['items_x_numberless'].to_list()[member])
    
    for tok in doc:
        print(tok, tok.pos_)
    print()

sevimli PROPN
kefir ADJ

sandora PROPN
yashil PROPN
alma PROPN

lays VERB
baharat NOUN

lays VERB
paprika NOUN

lays VERB
sir PROPN

doritos VERB
acili NOUN

sevimli PROPN
kesmik PROPN
gaysili PROPN

sevimli ADJ
xama PROPN
kaunas NOUN

sevimli ADJ
urekli ADJ
pendir NOUN

sevimli PROPN
kefir ADJ



In [700]:
import yake

In [800]:
kw_extractor = yake.KeywordExtractor(
    n=2,
    dedupLim=0.5,
    dedupFunc='sequencematcher',
    windowsSize=2,
    top=3,
    features=None
)

for job in comp_scored.sample(50)['items_x_numberless']:
    keywords = kw_extractor.extract_keywords(job)
    print(job)
    for kw in keywords:
        print(kw)
    print('--')
    x = {kw[0]: kw[1] for kw in keywords}
    # print({k: v for k, v in sorted(x.items(), key=lambda item: item[1])})
    print(min(x, key=x.get))
    fltr = kw_extractor.extract_keywords((min(x, key=x.get)))
    fltr_dct = {kw[0]: kw[1] for kw in fltr}
    print(max(fltr_dct, key=fltr_dct.get))
    print()

sonti albali nektari
('sonti albali', 0.04940384002065631)
('albali nektari', 0.04940384002065631)
('sonti', 0.15831692877998726)
--
sonti albali
sonti

turist klassik
('turist klassik', 0.02570861714399338)
('turist', 0.15831692877998726)
('klassik', 0.15831692877998726)
--
turist klassik
turist

elvan toffix mix
('elvan toffix', 0.04940384002065631)
('toffix mix', 0.04940384002065631)
('elvan', 0.15831692877998726)
--
elvan toffix
elvan

sevimli xama kaunasskaya
('sevimli xama', 0.04940384002065631)
('xama kaunasskaya', 0.04940384002065631)
('xama', 0.29736558256021506)
--
sevimli xama
xama

viko portagal
('viko portagal', 0.02570861714399338)
('viko', 0.15831692877998726)
--
viko portagal
viko

sev yumru duyu
('sev yumru', 0.04940384002065631)
('yumru duyu', 0.04940384002065631)
('sev', 0.15831692877998726)
--
sev yumru
sev

lays sir
('lays sir', 0.02570861714399338)
('lays', 0.15831692877998726)
('sir', 0.15831692877998726)
--
lays sir
lays

ulker icimino alma
('ulker icimino', 0.0

In [803]:
def get_2main_word(word: str):
    keywords = kw_extractor.extract_keywords(word)
    x = {kw[0]: kw[1] for kw in keywords}
    if len(x) > 0:
        return min(x, key=x.get)
    else:
        return ''

def get_1main_word(words: str):
    if len(words) > 0:
        fltr = kw_extractor.extract_keywords(words)
        fltr_dct = {kw[0]: kw[1] for kw in fltr}
        return max(fltr_dct, key=fltr_dct.get)
    else:
        return ''

In [805]:
for item in comp_scored.sample(50)['items_x_numberless']:
    words = get_2main_word(item)
    word = get_1main_word(words)
    print(words)
    print(word)
    print()

lays paprika
lays

lays stix
lays

sevimli canax
canax

ulker pendir
ulker

sevimli sud
sud

fersan detox
fersan

pinar milfoy
pinar

sevimli xama
xama

zolotoy petusok
zolotoy

suxari biftekli
suxari

pinar peynir
pinar

pinar peynir
pinar

icimino sud
sud

suxari biftekli
suxari

bebeto spaghetti
bebeto

pinar peynir
pinar

lays rebrishki
lays

karsa klas
karsa

suxari biftekli
suxari

osmanli lokum
lokum

tadim antep
tadim

tadim aycekirdek
tadim

today goldies
today

pamuk tek
tek

elvan today
elvan

ulker puding
ulker

pinar beyaz
pinar

lays shashlik
lays

sevimli xama
xama

pinar krema
pinar

frico gouda
frico

elvan today
elvan

semeyniy budj
budj

doktor milk
milk

lays ridged
lays

milla ayran
milla

sev manni
sev

bebeto boyuk
bebeto

fanni vanilli
fanni

superfresh ton
ton

lays rebrishki
lays

doktor milk
milk

pinar kofte
pinar

bagdat citir
bagdat

pinar peynir
pinar

pinar sosis
pinar

suxari xrus
xrus

lays ridged
lays

doritos acili
acili

lays ridged
lays



In [807]:
comp_scored['main_2word'] = comp_scored['items_x_numberless'].apply(lambda x: get_2main_word(x))
comp_scored['main_1word'] = comp_scored['main_2word'].apply(lambda x: get_1main_word(x))

In [810]:
comp_scored[['main_1word', 'main_2word', 'items_x_numberless', 'items_x']].sample(10)

,main_1word,main_2word,items_x_numberless,items_x
15491048,gatig,sevimli gatig,sevimli gatig suse,sevimli dad gatig suse 0.5 pct 500 gr 1 x 12
12636727,kefir,sevimli kefir,sevimli kefir kaunas,sevimli dad kefir kaunas 3.5 pct 500 gr 1 x 30
14660363,asci,asci manni,asci manni,asci manni 800 gr 1 x 12
16342964,port,sochnaya port,sochnaya port mango,sochnaya port mango mandar 0.95 lt 1 x 12
20371986,lays,lays,lays t?ndi̇rd?,lays t?ndi̇rd? ?t 90 gr 1 x 25
11120463,qati,qlavprodukt qati,qlavprodukt qati sud,qlavprodukt qati sud paket sokolad 250 qr 1 x 30
6534023,pashtet,qlavprodukt pashtet,qlavprodukt pashtet mal,qlavprodukt pashtet mal 240 gr 1 x 20
6048491,dilimli,president dilimli,president dilimli maasdam,president dilimli maasdam 150 gr 46 pct
3622304,sud,sevimli sud,sevimli sud goycay,sevimli dad sud 1 lt 2.5 pct 1 x 12 goycay
4169097,berrak,berrak biber,berrak biber tursusu,berrak biber tursusu aci 370 ml biberiye 1 x 12


In [814]:
comp_scored.groupby(by=['main_2word', 'items_x_numberless', 'Product or Service Name_x', 'Product or Service Name_y', 'unit_common_str']).agg({'quant_in':'sum','quant_out':'sum'}).sort_values('unit_common_str')

quant_in  \
main_2word     items_x_numberless   Product or Service Name_x                          Product or Service Name_y                 unit_common_str             
president xama president xama       PRESIDENT XAMA 15% 350 GR                          Xama "President" smetana 15% 350gr x 8    cl adet ədəd       1634.0   
sevimli kere   sevimli kere yag     SEVIMLI DAD KERE YAG 200 GR 1X24                   SEVIMLI DAD QAYMAQ 200 GR                 eded              34690.0   
sevimli kefir  sevimli kefir kaunas SEVIMLI DAD KEFIR KAUNAS 3.5% 500 GR 1X30          SEVIMLI DAD KEFIR KAUNAS 3.5% 500 GR 1X30 eded              91221.0   
               sevimli kefir hdy    SEVIMLI DAD KEFIR 2.5% 900GR AL 2.5% 500GR HDY 1X1 SEVIMLI DAD KEFIR 2.5% 500 GR             eded                 76.0   
                                    SEVIMLI DAD KEFIR 2.5% 500GR 1 AL 1 HDY 1X1        SEVIMLI DAD KEFIR 2.5% 500 GR             eded                  3.0   
...                                                                                                                                                    ...   
linex          linex                Linex N16 03.2022                                  LINEX 16 HGC                              ədəd                 12.0   
                                    Linex N16 08.2021 (357)                            LINEX 16 HGC                              ədəd                127.0   
lomilan        lomilan              Lomilan 10mg N10 04.2023 (336)                     LOMILAN 10 MG N 10                        ədəd                  8.0   
nakom          nakom                Nakom 250+25mg N100 09.2021 (90)                   NAKOM 250 MG+25 MG 100 TAB                ədəd                  6.0   
esomeprazol    esomeprazol          Esomeprazol 20mg N28 08.2021                       ESOMEPRAZOL 20 MG N 28                    ədəd                  1.0   

                                                                                                                                                   quant_out  
main_2word     items_x_numberless   Product or Service Name_x                          Product or Service Name_y                 unit_common_str              
president xama president xama       PRESIDENT XAMA 15% 350 GR                          Xama "President" smetana 15% 350gr x 8    cl adet ədəd        31536.0  
sevimli kere   sevimli kere yag     SEVIMLI DAD KERE YAG 200 GR 1X24                   SEVIMLI DAD QAYMAQ 200 GR                 eded             10565960.0  
sevimli kefir  sevimli kefir kaunas SEVIMLI DAD KEFIR KAUNAS 3.5% 500 GR 1X30          SEVIMLI DAD KEFIR KAUNAS 3.5% 500 GR 1X30 eded             35733000.0  
               sevimli kefir hdy    SEVIMLI DAD KEFIR 2.5% 900GR AL 2.5% 500GR HDY 1X1 SEVIMLI DAD KEFIR 2.5% 500 GR             eded                42922.0  
                                    SEVIMLI DAD KEFIR 2.5% 500GR 1 AL 1 HDY 1X1        SEVIMLI DAD KEFIR 2.5% 500 GR             eded                 3902.0  
...                                                                                                                                                      ...  
linex          linex                Linex N16 03.2022                                  LINEX 16 HGC                              ədəd                 1785.0  
                                    Linex N16 08.2021 (357)                            LINEX 16 HGC                              ədəd                 8925.0  
lomilan        lomilan              Lomilan 10mg N10 04.2023 (336)                     LOMILAN 10 MG N 10                        ədəd                 1344.0  
nakom          nakom                Nakom 250+25mg N100 09.2021 (90)                   NAKOM 250 MG+25 MG 100 TAB                ədəd                 2880.0  
esomeprazol    esomeprazol          Esomeprazol 20mg N28 08.2021                       ESOMEPRAZOL 20 MG N 28                    ədəd                  100.0  

[5147 rows x 2 columns]

In [778]:
import lda
import json
import logging
import collections
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


class StageLDA:
    def __init__(self, texts_data: pd.DataFrame, topics_number: int = 7):
        self.texts_data = texts_data
        self.topics_number = topics_number

    def run(self) -> str:
        anons_lda = collections.defaultdict(list)
        text_lda = collections.defaultdict(list)
        model = lda.LDA(n_topics=self.topics_number, random_state=123)
        logging.getLogger("lda").setLevel(logging.ERROR)

        keys = list(self.texts_data.index)
        corpus = self.texts_data['items_x_numberless'].to_list()

        vec = CountVectorizer(analyzer='word', ngram_range=(1, 3))
        x = vec.fit_transform(corpus)

        model.fit(x)
        doc_topic = model.transform(x)

        for i, memberships in enumerate(doc_topic):
            indices = np.argsort(memberships)[::-1]
            anons_lda[int(indices[0])].append(keys[i])
            text_lda[int(indices[0])].append(corpus[i])

        res = json.dumps(text_lda, indent=1, ensure_ascii=False)

        return json.loads(res)


In [785]:
def get_analyzed_texts(texts_data: pd.DataFrame):
    topics_number = 25 # int(texts_data.shape[0] / 100)
    stage = StageLDA(texts_data=texts_data, topics_number=topics_number)
    res = stage.run()

    for x in range(topics_number):
        print(x)
        [print(x) for x in set(res.get(str(x)))]


In [786]:
get_analyzed_texts(comp_scored.sample(1000))

0
suxari sir
lays sir
lays rebrishki qril
beliy qorod sud
lays doner
muratbey taze kasar
icimino sud bananli
beliy qorod qaymaq
1
elvan today donut
elvan king tat
groprinosin mg.ml
elvan toffix mix
elvan today croissant
elvan barcake orman
elvan today goldies
elvan today snowball
elvan today soko.karamel.mini
suxari toyuq qanadlari
elvan today roll
elvan anl toy
elvan today aloha
elvan kokos
elvan jelaxy fizzy
elvan anl champion
2
fora yesil zeytin
qlavprodukt adjika pikatnaya
deste siyah zeytin
suxari bavarskie kolbaski
fora cesnili yesil
fora siyah zeytin
sevimli qatiq suse
qlavprodukt shprot easy
qlavprodukt qabaq kurusu
qlavprodukt adjika kavkazskaya
qlavprodukt adjika aromatnaya
3
sevimli gatig
smuzi angry birds
sevimli gatig suse
angry birds gilemeyve
superfresh taze fasulye
karsa cikobest hindis.cevizli
angry birds visne
zi̇non alfasipermetrin
sandora mors gilemeyve
sevimli dovga
4
elvan shock sokolad
royal holland pendiri
ulker icimino ciyelek
bebeto laces
galbani maskarpone pe

In [460]:
df_incoming[df_incoming['Product or Service Name'] == 'SEVIMLI DAD KESMIK GAYSILI 9% 180 GR 1x1']['unit'].unique()

array(['eded'], dtype=object)

In [461]:
df_outcoming[df_outcoming['Product or Service Name'] == 'SEVIMLI DAD KESMIK ALBALILI 0% 180 GR1x1']['unit'].unique()

array(['adet.ml', 'eded', 'adet.lt'], dtype=object)

In [49]:
import pickle

In [51]:
with open('res_int.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(res_int, outp, pickle.HIGHEST_PROTOCOL)

In [52]:
with open('res_ext.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(res_ext, outp, pickle.HIGHEST_PROTOCOL)

In [53]:
with open('score.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(score, outp, pickle.HIGHEST_PROTOCOL)